<a href="https://colab.research.google.com/github/andrelumesi/sinf0071-20232/blob/main/atividades/ThreadsC_C%2B%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalhando com Threads no C/C++

##1.Instruções Gerais.
1. As respostas das questões dos tutoriais devem ser colocadas no github do grupo, na pasta atividades\atividadesthread.

2. O prazo para entrega desta atividade é até 11/11.

3. Em caso de atraso, o grupo sofrerá penalização na nota final de acordo com a quantidade de dias adicionais necessários para postagem da atividade em seu repositório.

## 2.PosixThreads

- Biblioteca de Threads no espaço do usuário.

- Oferece funções e tipos de dados para a criação, sincronização e comunicação entre threads.
- Criação de Threads:
  - O processo de criação de uma nova thread é realizado pela função pthread_create.
  
  ```
  int pthread_create(pthread_t *thread, const pthread_attr_t *attr,
                          void *(*start_routine) (void *), void *arg);
  ```
  Ele utiliza quatro parâmetros:


1. Ponteiro para thread
2. Um struct que define características da thread (atributos da thread)
3. Ponteiro para função que a thread vai executar
4. Parâmetros que serão usados por esta função.

A seguir, um exemplo de código que demonstra o uso de ```pthread_create```. A diretiva ```%%writefile``` é usada para criar o arquivo ```criandothreads.c```. Execute para gerar o arquivo ```criandothreads.c```.

  


In [18]:
%%writefile criandothreads.c
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

int f; // Dado compartilhado entre as threads
void *funcaof(void *param); //Assinatura da função que será executada pela thread.

int main(int argc, char *argv[]){
    if (argc != 2 && atoi(argv[1]) < 0) {
	    fprintf(stderr,"sintaxe: ./criandothreads <valor inteiro maior que 0>\n");
	    return -1;}

    pthread_t thread1; //cria variável do tipo thread
    pthread_attr_t attr; // cria variável do tipo atributo de thread
    pthread_attr_init(&attr); // inicializa attr com valores padroes

    pthread_create(&thread1,&attr,funcaof,argv[1]); // cria a thread

    pthread_join(thread1,NULL);
    printf("f = %d\n",f);
}

void *funcaof(void *param) {
    int i, ultimo = atoi(param);
    f = 0;

	if (ultimo > 0)
		for (i = 1; i <= ultimo; i++)
			f += i;

  pthread_exit(0);
}

Overwriting criandothreads.c


Para executar os comandos presentes no shell, utilizamos ```!``` antes do comando almejado.
A seguir, chamamos o gcc para compilar o código, gerando como saída o executável ```criandothreadsbin```.  Em seguida, usamos ``` ./criandothreadsbin 10 ``` para executação do programa.

In [19]:
!gcc criandothreads.c -o criandothreadsbin

In [20]:
! ./criandothreadsbin 10

f = 55


### Questão 01. Mude o valor na chamada ``` criandothreadsbin ``` para ``` 20 ```. Qual foi o novo valor devolvido para ```f```?



### Questão 02. O que faz a função ```funcaof```?

###Questão 03. Crie uma nova funcao chamada ```funcaok``` e a variável global booleana ```k```. Esta funcao deverá executar em uma nova thread, denominada, ```thread2```.  ```funcaok``` apresenta mesma assinatura de ```funcaof```. No entanto, ```funcaok``` avalia se o valor passado como parâmetro é primo. Caso seja primo, ``` k = true ```, caso contrário, ``` k = false ```. Informe no método main (através de print) se o valor associado a k após execução de ```funcaok``` é ``` true ``` ou ``` false ```.

###Questão 04. Na caixa de código abaixo, execute o comando ```!unimize```, necessário para realização da Questões 05  e 06.

In [26]:
! unminimize

This system has been minimized by removing packages and content that are
not required on a system that users do not log into.

This script restores content and packages that are found on a default
Ubuntu server system in order to make this system more suitable for
interactive use.

Reinstallation of packages may fail due to changes to the system
configuration, the presence of third-party packages, or for other
reasons.

This operation may take some time.

Would you like to continue? [y/N] y



###Questão 05 - Na caixa de código a seguir, utilize o comando ``` man ```  para ```pthread_create``` e informe em que casos uma nova thread é finalizada.

PTHREAD_CREATE(3)                   Linux Programmer's Manual                   PTHREAD_CREATE(3)

NNAAMMEE
       pthread_create - create a new thread

SSYYNNOOPPSSIISS
       ##iinncclluuddee <<pptthhrreeaadd..hh>>

       iinntt pptthhrreeaadd__ccrreeaattee((pptthhrreeaadd__tt **_t_h_r_e_a_d,, ccoonnsstt pptthhrreeaadd__aattttrr__tt **_a_t_t_r,,
                          vvooiidd **((**_s_t_a_r_t___r_o_u_t_i_n_e)) ((vvooiidd **)),, vvooiidd **_a_r_g));;

       Compile and link with _-_p_t_h_r_e_a_d.

DDEESSCCRRIIPPTTIIOONN
       The  pptthhrreeaadd__ccrreeaattee() function starts a new thread in the calling process.  The new thread
       starts execution by invoking _s_t_a_r_t___r_o_u_t_i_n_e(); _a_r_g  is  passed  as  the  sole  argument  of
       _s_t_a_r_t___r_o_u_t_i_n_e().

       The 

###Questão 06. Investigue em https://docs.oracle.com/cd/E19455-01/806-5257/attrib-34610/index.html os valores associados por padrão pelo método ```pthread_attr_init``` a varáveis do tipo ```pthread_attr_t```. Apresente o significado de cada um deles.

###Questão 07. Na caixa de código a seguir, utilize o comando ``` man ``` para ``` pthread_join ```  e informe para que serve este comando.

## 3.Threads em modo Kernel
- No linux, threads são tasks
  - Tanto threads como processos são tarefas. No entanto, thread é uma tarefa que compartilha recursos.
  - A criação de threads é feita através da função clone():
  ```
    int clone(int (*fn)(void *), void *stack, int flags, void *arg)
  ```
  
    onde:
    - fn:ponteiro para função
    - stack: ponteiro para o topo da stack
    - flags: parâmetros que sinalizam o tipo de thread a ser gerado.
    - arg: parâmetro da função

- A seguir, temos código que apresenta exemplo de uso de clone. Execute-o para gerar o arquivo ```clone.c```.


In [30]:
%%writefile clone.c
#define _GNU_SOURCE
#include <sched.h>
#include <stdio.h>
#include <stdlib.h>
#include <sys/wait.h>
const int STACK_SIZE=65536; // Tamanho da Stack (65536 bytes ou 64 KiB).


// Codigo da Thread
int thread_code(void *arg){
   int i = 0;
   int n = atoi(arg);
   printf("Executando na tarefa filha %d \n", n);
   for ( i = 1 ; i <= 10 ; i++ )
      printf("\t thread(%d) = %d * %d = %d\n", n, n, i, (n*i));
   printf("\n");
   return 0;
}

void main(int argc, char *argv[]){
  // Ponteiro para a base e topo da Stack
   void* stack_base[argc-1];
   void* stack_top[argc-1];

   // ID de cada thread
   int  pid[argc-1];

   for (int j = 1; j < argc; j++){
     // Alocacao das Stacks de cada thread
   	 stack_base[j-1] = malloc(STACK_SIZE);
     if(stack_base[j-1]==NULL) {
        perror("malloc");
        exit(1);
     }
         // Topo da stack = base + tamanho da stack
     stack_top[j-1] = stack_base[j-1] + STACK_SIZE;
   }

   printf ("Foi alocado espaco para (%d) threads\n", argc-1);
   for (int j = 1; j < argc; j++){
        // Criando Threads
        pid[j-1] = clone(thread_code, stack_top[j-1], CLONE_VM | CLONE_FS | CLONE_FILES | CLONE_SIGHAND, argv[j]);
        if (pid[j-1] < 0) {
            printf("Erro: Nao foi possivel criar a thread.\n");
            exit(EXIT_FAILURE);
        }
   }

   // Aguarda término das threads
   for (int j = 1; j < argc; j ++)
      wait(&pid[j-1]);

   // Libera stack de cada thread
   for (int j = 1; j < argc; j++){
     free(stack_base[j-1]);
   }
   printf("Threads filhas finalizadas.\n");
}


Writing clone.c


Para executar os comandos presentes no shell, utilizamos ```!``` antes do comando almejado.
A seguir, chamamos o gcc para compilar o código, gerando como saída o executável ```clonebin```.  Em seguida, usamos ```./clonebin 1 2``` para executação do programa.

In [31]:
!gcc clone.c -o clonebin

In [33]:
! ./clonebin 1 2

Foi alocado espaco para (2) threads
Processos filho finalizado.
Executando na tarefa filha 2 
	 thread(2) = 2 * 1 = 2
	 thread(2) = 2 * 2 = 4
	 thread(2) = 2 * 3 = 6
	 thread(2) = 2 * 4 = 8
	 thread(2) = 2 * 5 = 10
	 thread(2) = 2 * 6 = 12
	 thread(2) = 2 * 7 = 14
	 thread(2) = 2 * 8 = 16
	 thread(2) = 2 * 9 = 18
	 thread(2) = 2 * 10 = 20

Executando na tarefa filha 1 
	 thread(1) = 1 * 1 = 1
	 thread(1) = 1 * 2 = 2
	 thread(1) = 1 * 3 = 3
	 thread(1) = 1 * 4 = 4
	 thread(1) = 1 * 5 = 5
	 thread(1) = 1 * 6 = 6
	 thread(1) = 1 * 7 = 7
	 thread(1) = 1 * 8 = 8
	 thread(1) = 1 * 9 = 9
	 thread(1) = 1 * 10 = 10



###Questão 08. O que foi impresso ao final da execução de ``` ./clonebin 1 2 ``` ? Na linha de código abaixo faça uma nova execução, chamando o programa da seguinte forma: ``` ./clonebin 5 6 7 8 ```. O que foi impresso?

In [36]:
! ./clonebin 5 6 7 8

Foi alocado espaco para (4) threads
Processos filho finalizado.
EEExecutando na tarefa filha 5 
	 thread(5) = 5 * 1 = 5
	 thread(5) = 5 * 2 = 10
	 thread(5) = 5 * 3 = 15
	 thread(5) = 5 * 4 = 20
	 thread(5) = 5 * 5 = 25
	Executando na tarefa filha 8 
	 thread(8) = 8 * 1 = 8
	 thread(8) = 8 * 2 = 16
xecutando na tarefa filha 6 
		 thread(8) = 8 * 3 = 24
 thread(6) = 6 * 1 = 6
		 thread(8) = 8 * 4 = 32
 thread(6) = 6 * 2 = 12
		 thread(8) = 8 * 5 = 40
 thread(6) = 6 * 3 = 18
		 thread(8) = 8 * 6 = 48
 thread(6) = 6 * 4 = 24
			 thread(8) = 8 * 7 = 56
 thread(5) = 5 * 6 = 30
 thread(6) = 6 * 5 = 30
	 thread(5) = 5 * 7 = 35
	 thread(5) = 5 * 8 = 40
	 thread(5) = 5 * 9 = 45
	 thread(5) = 5 * 10 = 50


Executando na tarefa filha 7 
		 thread(6) = 6 * 6 = 36
	 thread(6) = 6 * 7 = 42
	 thread(6) = 6 * 8 = 48
	 thread(8) = 8 * 8 = 64
 thread(6) = 6 * 9 = 54
		 thread(8) = 8 * 9 = 72
 thread(6) = 6 * 10 = 60

	 thread(8) = 8 * 10 = 80


 thread(7) = 7 * 1 = 7
	 thread(7) = 7 * 2 = 14
	 thread(7)

### Questão 09. Execute por mais vezes a chamada ``` ./clonebin 5 6 7 8 ```. As respostas de cada thread sempre são impressas na mesma ordem ou há mistura. ? Caso ocorra mistura, qual o motivo que leva a essa mistura?

###Questão 10. Explique o que está sendo feito na função ``` int thread_code(void *arg)``` (linhas 11-19).


###Questão 11. Na caixa de código a seguir, execute o comando ``` man ``` para a função ``` clone ```. Extraia do manual de ``` clone ```:
1. Para que serve a função clone?
2. Para que serve as constantes ``` CLONE_VM ```, ``` CLONE_FS ```, ``` CLONE_FILES ```, ``` CLONE_SIGHAND``` usadas pelo código na linha 43.

###Questão 12. Para que serve a região de memória denominada stack? Threads possuem stack. Elas são individuais ou compartilhadas, justifique.

###Questão 13. Em clone.c qual o tamanho da stack adotada por cada thread? Explique o que acontece nas linhas [22-38] do código.

## 4.Criando processos filhos
- Um processo é um programa em execução.
- A criação de processos no Linux, pode ser feita através do método fork(), presente em unistd.h
  - ``` pid_t fork(void); ```
- fork cria um novo processo, através da duplicação do processo que gerou a chamada. O novo processo é denominado ﬁlho. O antigo,pai.
- Filho e pai executam em espaços de memória diferentes, porém apresentam o mesmo código.
- A seguir, apresentamos um exemplo de código ilustrando seu uso. Execute-o para gerar o arquivo ``` exemplo01.c ```

In [48]:
%%writefile exemplo01.c
#include <stdio.h>
#include <sys/types.h>
#include <unistd.h>
#include <wait.h>

int valor = 5;

int main(){
	pid_t pid;

	pid = fork();

	if (pid == 0) {
		valor += 15;
    printf ("Processo filho: valor = %d\n",valor);
		return 0;
	}
	else if (pid > 0) {
		wait(NULL);
		printf ("Processo pai: valor = %d\n",valor); /* LINE A */
		return 0;
	}
}

Overwriting exemplo01.c


Para executar os comandos presentes no shell, utilizamos ```!``` antes do comando almejado.
A seguir, chamamos o gcc para compilar o código, gerando como saída o executável ```clonebin```.  Em seguida, usamos ```./clonebin 1 2``` para executação do programa.

In [49]:
!gcc exemplo01.c -o exemplo01bin

In [50]:
!./exemplo01bin

Processo filho: valor = 20
Processo pai: valor = 5


##Questão 14. Porque o processo pai e o processo filho apresentam valores diferentes para a variável global valor?

###Questão 15. Na caixa de código abaixo, use o comando ``` man ``` para consultar a função ``` fork ```. Qual o valor que ``` fork ``` devolve para o processo pai e para o processo filho?

###Questão 16. ``` getpid ``` é uma função que devolve o ``` process id ``` (identificador único) do processo em execução. Baseado nessa informação, execute as seguintes caixas de código para geração do arquivo ``` exemplo02.c ```, compilação e execução.

In [51]:
%%writefile exemplo02.c
#include <stdio.h>
#include <unistd.h>

int main()
{
	printf("%d\n",getpid());
	fork();
	printf("%d\n",getpid());


	fork();
	printf("%d\n",getpid());

	fork();
	printf("%d\n",getpid());

	return 0;
}

Writing exemplo02.c


In [52]:
!gcc exemplo02.c -o exemplo02bin

In [53]:
!./exemplo02bin

72770
72770
72770
72770
72773
72772
72772
72771
72771
72771
72774
72776
72775
72775
72777


###Questão 17. Baseado na execução da Questão 16, o que significam os números que foram impressos? Quantos números diferentes temos após a execução do código? Quantos processos foram criados durante a execução desse código?

###Questão 18. É possível associar uma fórmula que informe a quantidade de processos criados a depender da quantidade de forks? Considere ``` n```, como sendo a quantidade de forks e ``` nps ```, como sendo a quantidade de processos criados.

###Questão 19. Use as caixas de código a seguir para modificar, compilar e executar ``` 10processos.c ```. Ao final da execução de ``` 10processos.c ```, devem ser gerados 10 processos.

In [ ]:
%%writefile 10processos.c
#include <stdio.h>
#include <unistd.h>

int main()
{
	printf("%d\n",getpid());
	fork();
	printf("%d\n",getpid());


	fork();
	printf("%d\n",getpid());

	fork();
	printf("%d\n",getpid());

	return 0;
}

In [ ]:
!gcc 10processos.c -o 10processosbin

In [ ]:
!./10processosbin